In [1]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [2]:
from rpy2.robjects import r
r.data('iris')
df_iris = pandas2ri.ri2py(r.data('iris'))

In [3]:
import pandas.rpy.common as com
infert = com.load_data('infert')
infert.head()

/Users/yuewang/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
  if __name__ == '__main__':


,education,age,parity,induced,case,spontaneous,stratum,pooled.stratum
1,0-5yrs,26.0,6.0,1.0,1.0,2.0,1,3.0
2,0-5yrs,42.0,1.0,1.0,1.0,0.0,2,1.0
3,0-5yrs,39.0,6.0,2.0,1.0,0.0,3,4.0
4,0-5yrs,34.0,4.0,2.0,1.0,0.0,4,2.0
5,6-11yrs,35.0,3.0,1.0,1.0,1.0,5,32.0


In [4]:
from numpy import *
import scipy as sp
import pandas as pd
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import pandas.rpy.common as com

In [5]:
stats = importr('stats')
base = importr('base')
datasets = importr('datasets')
graph = importr('graphics')
forecast = importr('forecast')
fortify = importr('ggfortify')

In [6]:
ro.r('library(forecast)')

array(['ggfortify', 'ggplot2', 'forecast', 'timeDate', 'zoo', 'tools',
       'stats', 'graphics', 'grDevices', 'utils', 'datasets', 'methods',
       'base'], 
      dtype='|S9')

In [7]:
count = range(1, 51)
df = pd.DataFrame(index = pd.date_range('2016-01-01', '2016-02-19'), data = count, columns = ['count'])
# df['count'] = count
# df['date'] = pd.date_range('2016-01-01', '2016-02-19')
# df.set_index('date', inlace = True)
# df.sort_index(inplace = True)
pandas2ri.activate()
r_timeseries = pandas2ri.py2ri(df)

In [8]:
df.shape

(50, 1)

In [ ]:
rdf = pandas2ri.py2ri(df)
ro.globalenv['r_timeseries'] = rdf
pred = ro.r('as.data.frame(forecast(auto.arima(r_timeseries),h=5))')

In [ ]:
pred

In [ ]:
pred1 = ro.r('as.data.frame(forecast(auto.arima(r_timeseries), h=10))')
pred1

In [ ]:
ro.globalenv['r_timeseries']

In [ ]:
df = com.load_data('mtcars')

In [ ]:
rdf = com.convert_to_r_dataframe(df)

In [ ]:
formula = 'mpg ~ wt + cyl'

In [ ]:
fit_full = stats.lm(formula, data=rdf)
# print(base.summary(fit_full))

In [ ]:
volcanodust = ro.r.scan("http://robjhyndman.com/tsdldata/annual/dvi.dat", skip=1)

In [ ]:
volcanodustseries = ro.r.ts(volcanodust,start=ro.r.c(1500))

In [ ]:
# re = stats.pacf(volcanodustseries)

In [ ]:
import numpy as np
count = np.random.rand(214,1)
df = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-08-01'), data=count, columns=['count'])

In [ ]:
# df['count'].dropna()
df['count'].dropna().shape
df

In [ ]:
rdf = pandas2ri.py2ri(df['count'].dropna())
ro.globalenv['r_timeseries'] = rdf
pred = ro.r('as.data.frame(forecast(auto.arima(r_timeseries),h=1))')

In [ ]:
pred

In [ ]:
type(pred)

In [ ]:
df.shape[1]

In [ ]:
len(df['count'].shape)

In [ ]:
len(df.shape)

In [ ]:
count = np.random.rand(214,2)
df = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-08-01'), data=count, columns=['count1','count2'])

In [ ]:
df.dtypes

In [ ]:
df[['count1']].dropna().dtypes

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from BL_model_v2 import *
from BL_data_reader import *

In [ ]:
def FACTOR_EMA(df, half_life):
        return pd.ewma(df, halflife=half_life)

In [ ]:
view_config = {"MA5" : lambda df: FACTOR_EMA(df.shift(1).dropna(axis=0), 5),
"MA30" : lambda df: FACTOR_EMA(df.shift(1).dropna(axis=0), 30),
"MA120" : lambda df: FACTOR_EMA(df.shift(1).dropna(axis=0), 120)}


In [ ]:
DR_view = BLDR.view_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_return_w_industry.csv", view_config,                            error_periods=60, error_method = "rolling_window")
DR_factor_return = BLDR.factor_return_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_return_w_industry.csv",
                                                  look_back_periods=60)
DR_residual = BLDR.residual_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/residual.csv",
                                        look_back_periods=60, threshold = None, min_periods=2, diagonalized=True)
DR_residual.set_threshold(0.000001)
DR_factor_loading = BLDR.factor_loading_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_loading_w_industry.csv")

In [ ]:
data_reader_config = {'view': DR_view, #call method return estimation and standard error of each predictors
        'factor_return': DR_factor_return, #call method return prior mean and covariance of factors
            'residual': DR_residual,#call method return covariance of residuals
                'factor_loading': DR_factor_loading #call method return factor loadings of each stock, also return the return of next B day for each stock
                }
BLP = Black_Litterman_Portfolio(data_reader_config, 1, factor_diagonalized=False)
print BLP.view.methods

In [ ]:
target_date = datetime(2015,5,8)
EST, ERR = BLP.view('MA5',target_date)

In [ ]:
EST

In [ ]:
ERR

In [ ]:
# file_dir = "/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_return_w_industry.csv"
# _df_factor_return = pd.read_csv(file_dir, sep =',',index_col=0)
# _df_factor_return.index = _df_factor_return.index.map(lambda idx: datetime.strptime(idx,"%Y-%m-%d"))
# _df_factor_return.sort_index(inplace=True)

In [ ]:
# import pickle
# pickle.dump(_df_factor_return, open("factor_return.p", "wb" ) )

In [9]:
import pickle
_df_factor_return = pickle.load( open( "factor_return.p", "rb" ) )

In [10]:
_df_factor_return.iloc[:100,:].BETA

2006-01-03   -0.001133
2006-01-04    0.007586
2006-01-05    0.004873
2006-01-06    0.006084
2006-01-09    0.001911
2006-01-10   -0.002015
2006-01-11   -0.005629
2006-01-12   -0.001374
2006-01-13   -0.004801
2006-01-17    0.005985
2006-01-18    0.005533
2006-01-19   -0.010752
2006-01-20   -0.001251
2006-01-23    0.015835
2006-01-24   -0.001139
2006-01-25    0.009916
2006-01-26    0.001748
2006-01-27   -0.004918
2006-01-30    0.000441
2006-01-31   -0.001997
2006-02-01   -0.009870
2006-02-02   -0.002404
2006-02-03    0.002623
2006-02-06   -0.004417
2006-02-07    0.004299
2006-02-08   -0.006554
2006-02-09    0.003523
2006-02-10   -0.002989
2006-02-13    0.010988
2006-02-14    0.008018
                ...   
2006-04-13   -0.007315
2006-04-17    0.019592
2006-04-18    0.002268
2006-04-19    0.001658
2006-04-20   -0.008331
2006-04-21   -0.004343
2006-04-24   -0.003578
2006-04-25   -0.004077
2006-04-26   -0.001924
2006-04-27   -0.002858
2006-04-28   -0.005662
2006-05-01    0.002911
2006-05-02 

In [11]:
r_tmp = pandas2ri.py2ri(_df_factor_return.iloc[:200,:].BETA)

In [12]:
ro.globalenv['tmp'] = r_tmp

In [ ]:
# base.rm('tmp') ## Get rid of that global variables

In [ ]:
pred = ro.r('as.data.frame(forecast(auto.arima(tmp),h=5))')

In [ ]:
# _df_factor_return.iloc[:100,:].BETA
pred

In [ ]:
pred['Point Forecast'][0]

In [ ]:
# %matplotlib inline

In [ ]:
# _df_factor_return.iloc[:100,:].BETA.plot()

In [ ]:
#### use holt winter

In [ ]:
# (m <- HoltWinters(AirPassengers, seasonal = "mult"))

In [ ]:
# m = ro.r('as.data.frame(forecast(HoltWinters(tmp),12))')

In [ ]:
ro.globalenv['tmp']

In [ ]:
### have to convert tmp to a ts

In [ ]:
data_pd = _df_factor_return.iloc[200,:].BETA

In [ ]:
# data_pd.index[0], data_pd.index[-1] 

In [ ]:
# data_pd.index[0].year, data_pd.index[0].month, data_pd.index[0].day

In [ ]:
# ts1 = stats.ts(r_tmp, start=ro.r.c(data_pd.index[0].year, data_pd.index[0].month, data_pd.index[0].day
# ), end=ro.r.c(data_pd.index[1].year, data_pd.index[1].month, data_pd.index[1].day), frequency=365) 

In [ ]:
_df_factor_return.iloc[:200,:].BETA

In [ ]:
base.remove('r_tmp')

In [ ]:
ts1 = stats.ts(r_tmp, start=1, frequency=356) 

In [ ]:
ro.globalenv['ts1'] = ts1

In [ ]:
# hw <- HoltWinters(demand)
m = ro.r('as.data.frame(forecast(,1))')

In [ ]:
m

In [ ]:
##### VAR prediction

In [13]:
vars = importr("vars")

In [35]:
# data_pd_var.index

In [36]:
data_pd_var

,BETA,MOMENTUM,SIZE,VALUE,VOL,ConsumerDiscretionary,ConsumerStaples,Energy,Financials,HealthCare,Industrials,InformationTechnology,Materials,TelecommunicationServices,Utilities
2006-01-03,-0.001133,0.000223,0.001184,-0.000966,0.002665,0.003528,0.005669,0.011090,0.010016,0.009870,0.008881,0.010865,0.010585,0.008167,0.006355
2006-01-04,0.007586,-0.000938,0.000283,0.000634,0.001005,-0.005755,-0.009857,-0.024153,-0.006869,-0.006851,-0.010128,0.001687,-0.013826,-0.007670,-0.010478
2006-01-05,0.004873,0.000734,0.001563,-0.002175,-0.000213,0.000513,-0.000419,0.011861,0.001624,0.003856,0.005281,0.007377,0.009279,0.000625,0.004791
2006-01-06,0.006084,-0.000512,-0.000325,0.001041,0.002310,0.005520,0.002149,-0.007236,0.000032,0.005381,0.000890,-0.001432,-0.003378,-0.004361,-0.002595
2006-01-09,0.001911,0.001008,-0.001345,0.000613,0.001603,-0.001404,-0.000564,0.009569,0.002326,-0.002680,0.003928,0.000635,-0.001357,-0.003334,0.003034
2006-01-10,-0.002015,-0.000016,0.001755,-0.000209,0.001714,0.002867,0.000167,0.004769,0.004899,0.001411,0.002075,0.007763,-0.001123,0.011016,0.002184
2006-01-11,-0.005629,-0.000066,-0.000779,-0.000748,-0.000284,-0.002033,0.004199,0.001049,-0.000622,0.002719,0.000348,0.000144,-0.003576,-0.000560,0.003884
2006-01-12,-0.001374,0.000263,-0.000888,0.000213,0.000014,0.000983,0.001528,0.012244,0.002744,0.003315,0.001987,0.000850,0.006531,0.004180,0.006336
2006-01-13,-0.004801,0.001385,-0.000312,0.001130,-0.001097,-0.004333,-0.001051,0.021092,-0.002940,0.001653,-0.002902,-0.002414,0.004487,-0.008333,0.006350
2006-01-17,0.005985,-0.000668,-0.000990,-0.001270,-0.002273,-0.008428,-0.001198,-0.020875,-0.008963,-0.002295,-0.004488,-0.008605,-0.020411,-0.001611,-0.004673


In [ ]:
### using VAR from stats modles

In [60]:
import statsmodels.api as sm
import statsmodels.tsa.vector_ar.var_model as model
import statsmodels.tsa.vector_ar.util as util
import statsmodels.tools.data as data_util
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.dynamic import DynamicVAR

In [82]:
model = VAR(data_pd_var)

In [83]:
data_pd_var.shape

(200, 15)

In [76]:
# model.fit(2).summary()
data_pd_var.index

DatetimeIndex(['2006-01-03', '2006-01-04', '2006-01-05', '2006-01-06',
               '2006-01-09', '2006-01-10', '2006-01-11', '2006-01-12',
               '2006-01-13', '2006-01-17',
               ...
               '2006-10-04', '2006-10-05', '2006-10-06', '2006-10-09',
               '2006-10-10', '2006-10-11', '2006-10-12', '2006-10-13',
               '2006-10-16', '2006-10-17'],
              dtype='datetime64[ns]', length=200, freq=None)

In [118]:
var = DynamicVAR(data_pd_var, min_periods=data_pd_var.shape[0]-3, window_type='expanding')

In [119]:
data_pd_var.iloc[-2:,:]

,BETA,MOMENTUM,SIZE,VALUE,VOL,ConsumerDiscretionary,ConsumerStaples,Energy,Financials,HealthCare,Industrials,InformationTechnology,Materials,TelecommunicationServices,Utilities
2006-10-16,-0.005996,0.000514,-0.000165,-0.000984,0.000333,-0.000888,0.004278,-0.003818,0.003140,0.003926,-0.002971,-0.003441,0.00023,0.014507,0.009303
2006-10-17,-0.005911,-0.000528,-0.000332,-0.000698,-0.000879,0.006351,0.009444,0.002397,0.007658,0.011678,0.003086,-0.000685,0.00805,0.003803,0.010864


In [120]:
var.forecast(1)

,BETA,MOMENTUM,SIZE,VALUE,VOL,ConsumerDiscretionary,ConsumerStaples,Energy,Financials,HealthCare,Industrials,InformationTechnology,Materials,TelecommunicationServices,Utilities
2006-10-16,0.011270,0.003726,-0.007061,-0.008625,0.019438,0.002064,0.009668,0.001194,-0.015505,0.005627,-0.005516,0.002149,-0.002211,0.003664,0.005722
2006-10-17,0.001341,-0.002965,-0.003505,-0.000624,0.005174,0.000236,0.003355,-0.000597,-0.006241,-0.000978,0.003156,-0.000483,-0.000267,0.003665,0.002207


In [14]:
data_pd_var = _df_factor_return.iloc[:200,:]
r_tmp_var = pandas2ri.py2ri(data_pd_var)

In [15]:
data_pd_var.index

DatetimeIndex(['2006-01-03', '2006-01-04', '2006-01-05', '2006-01-06',
               '2006-01-09', '2006-01-10', '2006-01-11', '2006-01-12',
               '2006-01-13', '2006-01-17',
               ...
               '2006-10-04', '2006-10-05', '2006-10-06', '2006-10-09',
               '2006-10-10', '2006-10-11', '2006-10-12', '2006-10-13',
               '2006-10-16', '2006-10-17'],
              dtype='datetime64[ns]', length=200, freq=None)

In [16]:
ts1_var = stats.ts(r_tmp_var, start=1, frequency=356) 

In [17]:
ro.globalenv['ts1_var'] = ts1_var

In [18]:
ro.globalenv['r_tmp_var'] = r_tmp_var

In [ ]:
# print(r_tmp_var)

In [19]:
# ZOO = ro.r('zoo(r_tmp_var, order.by=as.Date(as.character(row.names(r_tmp_var)),format=\'%Y/%m/%d\'))')
ZOO = ro.r('zoo(r_tmp_var)')

In [20]:
ro.globalenv['tmp'] = ZOO

In [21]:
ro.globalenv['ZOO'] = stats.ts(ZOO)

In [23]:
print type(ro.globalenv['ZOO'])

<class 'rpy2.robjects.vectors.Matrix'>


In [24]:
print ro.globalenv['ZOO']

Time Series:
Start = 1 
End = 200 
Frequency = 1 
         Series 1      Series 2      Series 3      Series 4      Series 5
  1 -0.0011334058  2.231568e-04  1.184032e-03 -9.663541e-04  2.665475e-03
  2  0.0075863827 -9.377390e-04  2.826469e-04  6.335979e-04  1.004829e-03
  3  0.0048734598  7.341494e-04  1.562787e-03 -2.175152e-03 -2.134603e-04
  4  0.0060835749 -5.123766e-04 -3.251942e-04  1.041131e-03  2.310332e-03
  5  0.0019110080  1.008028e-03 -1.345387e-03  6.125731e-04  1.603273e-03
  6 -0.0020154915 -1.586674e-05  1.754559e-03 -2.086211e-04  1.713940e-03
  7 -0.0056294718 -6.617091e-05 -7.794577e-04 -7.479135e-04 -2.842824e-04
  8 -0.0013737138  2.631754e-04 -8.884500e-04  2.130032e-04  1.396768e-05
  9 -0.0048009972  1.384828e-03 -3.119958e-04  1.130314e-03 -1.097286e-03
 10  0.0059848520 -6.676838e-04 -9.895724e-04 -1.270115e-03 -2.273313e-03
 11  0.0055325627  8.043809e-04 -1.065276e-03 -2.370377e-03  2.049259e-03
 12 -0.0107518176 -3.572877e-05 -2.420375e-03  4.361507e-05 -8

In [25]:
"as.data.frame(fortify(predict(VAR(ZOO,step), n.ahead = lag)))".replace("step", "1").replace("lag", "1")

'as.data.frame(fortify(predict(VAR(ZOO,1), n.ahead = 1)))'

In [31]:
m1 = ro.r("as.data.frame(fortify(predict(VAR(ZOO,step), n.ahead = lag)))".replace("step", "1").replace("lag", "1"))

In [32]:
m1

,Index,Series.1,Series.2,Series.3,Series.4,Series.5,Series.6,Series.7,Series.8,Series.9,...,Series.13.upper,Series.13.CI,Series.14.fcst,Series.14.lower,Series.14.upper,Series.14.CI,Series.15.fcst,Series.15.lower,Series.15.upper,Series.15.CI
1,1.0,-0.001133,0.000223,0.001184,-0.000966,0.002665,0.003528,0.005669,0.011090,0.010016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,0.007586,-0.000938,0.000283,0.000634,0.001005,-0.005755,-0.009857,-0.024153,-0.006869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,0.004873,0.000734,0.001563,-0.002175,-0.000213,0.000513,-0.000419,0.011861,0.001624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,0.006084,-0.000512,-0.000325,0.001041,0.002310,0.005520,0.002149,-0.007236,0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,0.001911,0.001008,-0.001345,0.000613,0.001603,-0.001404,-0.000564,0.009569,0.002326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,-0.002015,-0.000016,0.001755,-0.000209,0.001714,0.002867,0.000167,0.004769,0.004899,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,-0.005629,-0.000066,-0.000779,-0.000748,-0.000284,-0.002033,0.004199,0.001049,-0.000622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,-0.001374,0.000263,-0.000888,0.000213,0.000014,0.000983,0.001528,0.012244,0.002744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,-0.004801,0.001385,-0.000312,0.001130,-0.001097,-0.004333,-0.001051,0.021092,-0.002940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10.0,0.005985,-0.000668,-0.000990,-0.001270,-0.002273,-0.008428,-0.001198,-0.020875,-0.008963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
ro.r('as.data.frame(fortify(predict(VAR(ZOO,p=1, type="const"), n.ahead=1)))')    
# as.data.frame(fortify(vars.predict(VAR(var,p=1,type="const"), n.ahead=1)))

,Index,Series.1,Series.2,Series.3,Series.4,Series.5,Series.6,Series.7,Series.8,Series.9,...,Series.13.upper,Series.13.CI,Series.14.fcst,Series.14.lower,Series.14.upper,Series.14.CI,Series.15.fcst,Series.15.lower,Series.15.upper,Series.15.CI
1,1.0,-0.001133,0.000223,0.001184,-0.000966,0.002665,0.003528,0.005669,0.011090,0.010016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,0.007586,-0.000938,0.000283,0.000634,0.001005,-0.005755,-0.009857,-0.024153,-0.006869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,0.004873,0.000734,0.001563,-0.002175,-0.000213,0.000513,-0.000419,0.011861,0.001624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,0.006084,-0.000512,-0.000325,0.001041,0.002310,0.005520,0.002149,-0.007236,0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,0.001911,0.001008,-0.001345,0.000613,0.001603,-0.001404,-0.000564,0.009569,0.002326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,-0.002015,-0.000016,0.001755,-0.000209,0.001714,0.002867,0.000167,0.004769,0.004899,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,-0.005629,-0.000066,-0.000779,-0.000748,-0.000284,-0.002033,0.004199,0.001049,-0.000622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,-0.001374,0.000263,-0.000888,0.000213,0.000014,0.000983,0.001528,0.012244,0.002744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,-0.004801,0.001385,-0.000312,0.001130,-0.001097,-0.004333,-0.001051,0.021092,-0.002940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10.0,0.005985,-0.000668,-0.000990,-0.001270,-0.002273,-0.008428,-0.001198,-0.020875,-0.008963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
as.data.frame(forecast(HoltWinters(tmp,gamma = FALSE),h=1))
as.data.frame(fortify(predict(VAR(var_tmp,1), n.ahead=1)))

In [ ]:
count = np.random.rand(214, 2)
df = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-08-01'), data=count, columns=['count1','count2'])

In [ ]:
data_pd_var[['BETA','MOMENTUM']].values

In [ ]:
# data_pd_var.columns = None
test_matrix = data_pd_var.as_matrix()

In [ ]:
test_matrix = test_matrix.T

In [ ]:
tmp2 = pd.DataFrame(test_matrix, columns = None, index = data_pd_var.index)

In [ ]:
ts1_var = stats.ts(tmp2, start=1, frequency=356) 

In [ ]:
ro.globalenv['ts1_var'] = ts1_var

In [ ]:
a = 2

In [ ]:
"as.data.frame(fortify(predict(VAR(ts1_var,AAA), n.ahead = 1)))".replace("AAA", str(a))

In [ ]:
m1 = ro.r("as.data.frame(fortify(predict(VAR(ts1_var,AAA), n.ahead = 1)))".replace("AAA", str(a)))

In [ ]:
print type(ro.r('\''+ str(r_tmp_var) + '\''))

In [ ]:
# print ro.globalenv['ts1_var']

In [ ]:
# m1

In [ ]:
m = ro.r('as.data.frame(fortify(predict(VAR(ts1_var,1), n.ahead = 1)))')

In [ ]:
ro.globalenv['m'] = m

In [ ]:
pydf = com.load_data('m')
pydf.columns

In [ ]:
pydf = pydf.filter(regex='fcst')
pydf

In [ ]:
ts1 = stats.ts(r_tmp, start=1, frequency=356) 

In [ ]:


_df_factor_return.iloc[:200,:]

In [ ]:
tmp = pd.DataFrame(index=_df_factor_return.index, columns=_df_factor_return.columns)

In [ ]:
tmp.iloc[:50, :] = _df_factor_return.iloc[:50, :]

In [ ]:
tmp.shape

In [ ]:
for col in tmp.columns:
    for row in tmp.iloc[2510:,:].index:
#         print (row, col)
        tmp.ix[row,[col]] = 2
tmp.ix[row,[col]]

In [ ]:
count = np.random.rand(100,1)
df = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-04-09'), data=count, columns=['count'])

In [ ]:
df

In [ ]:
tmp.ix[row,[col]]

In [ ]:
tmp.tail

In [ ]:
from statsmodels.tsa.arima_process import arma_generate_sample
import statsmodels.api as sm
import numpy as np

In [ ]:
res = sm.tsa.arma_order_select_ic(np.array(_df_factor_return.BETA), ic=['aic', 'bic'], trend='nc')

In [ ]:
raw_predictor = sm.tsa.ARMA(_df_factor_return.BETA, res.aic_min_order).fit()

In [ ]:
sm.tsa.x13_arima_select_order()

In [ ]:
res1 = sm.tsa.arima_order_select_ic(np.array(_df_factor_return.iloc[:100,:].BETA), ic=['aic', 'bic'], trend='nc')

In [ ]:
raw_predictor1 = sm.tsa.ARMA(_df_factor_return.iloc[:100,:].BETA, res1.bic_min_order).fit()

In [ ]:
_df_factor_return.iloc[:100,:].BETA

In [ ]:
import datetime
datetime.date(2006, 5, 26)

In [ ]:
raw_predictor1.predict('20060526','20060527', dynamic=True)

In [ ]:
predict_sunspots = raw_predictor1.predict('1990', '2012', dynamic=True)

In [ ]:
_df_factor_return.columns

In [ ]:
len(np.array(_df_factor_return.BETA))

In [ ]:
len(np.array(_df_factor_return.iloc[:100,:].BETA))

In [ ]:
res.aic_min_order

In [ ]:
res.bic_min_order

In [ ]:
_df_factor_return

In [ ]:
dta = sm.datasets.sunspots.load_pandas().data

In [ ]:
dta

In [ ]:
import pandas

In [ ]:
mdata = sm.datasets.macrodata.load_pandas().data

In [ ]:
dates = mdata[['year', 'quarter']].astype(int).astype(str)

In [ ]:
quarterly = dates["year"] + "Q" + dates["quarter"]

In [ ]:
from statsmodels.tsa.base.datetools import dates_from_str

In [ ]:
quarterly = dates_from_str(quarterly)

In [ ]:
mdata = mdata[['realgdp','realcons','realinv']]

In [ ]:
mdata.index = pandas.DatetimeIndex(quarterly)

In [ ]:
data = np.log(mdata).diff().dropna()

In [ ]:
model = VAR(data)

In [ ]:
from __future__ import print_function
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.graphics.api import qqplot

In [ ]:
print(sm.datasets.sunspots.NOTE)


In [ ]:
dta = sm.datasets.sunspots.load_pandas().data

In [ ]:
dta.index = pd.Index(sm.tsa.datetools.dates_from_range('1700', '2008'))
del dta["YEAR"]